# Now let's look at this data with FOLIUM and clustering analysis

https://alcidanalytics.com/p/geographic-heatmap-in-python

### FOLIUM IS PRETTY COOL BEACUSE IT MEANS NO API CALLS, OR OTHER HARD TO INTERACT WITH AND RELY ON BS

In [93]:
#!pip install folium
import folium
from folium.plugins import HeatMap
import folium.plugins
folium.plugins.MarkerCluster()
from folium import IFrame

In [2]:
#!/usr/bin/python
import json
import requests
import pandas as pd
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
import seaborn as sns


/Users/RoscoeBColtrane/anaconda3/lib/python3.7/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [4]:
bestPizza = pd.read_csv("../data/PizzaEssentials.csv",index_col=False)
bestPizza.head()

restPandI = pd.read_csv("r../data/restPandI.csv",index_col=False)
restPandI['Latitude']=restPandI['Latitude'].apply(pd.to_numeric, downcast='float', errors='coerce')
#restPandI.drop(['Unnamed: 0'],axis=1, inplace=True)
print(restPandI.shape)
restPandI.head()
list(restPandI)

allPizza = pd.read_csv("../data/restPizza.csv",index_col = None)
allPizza['Latitude']=allPizza['Latitude'].apply(pd.to_numeric, downcast='float', errors='coerce')
print(allPizza.shape)
allPizza.head()

subway = pd.read_csv("../data/subwayStations.csv", index_col=0)
print(subway.shape)
#restPandI

(31171, 12)
(31171, 12)
(473, 8)


# Now we'll use FOLIUM and GEOPANDAS to visualzie a heat map.  

## This heat map will show the health inspectio scores

In [5]:
# https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KernelDensity.html#sklearn.neighbors.KernelDensity
restPandI.dtypes
#restPandI.drop(['Unnamed: 0'],axis=1, inplace=True)

Unnamed: 0                 int64
Longitude                float64
Latitude                 float64
Name                      object
BORO                      object
BUILDING                  object
STREET                    object
ZIPCODE                  float64
CUISINE DESCRIPTION       object
SCORE                    float64
VIOLATION DESCRIPTION     object
GRADE                     object
dtype: object

In [9]:
# CREATING GOEDATAFRAMES WITH POINT COLUMNS THAT ARE A GEO SERIES

# ALL PIZZA RESTAURANTS WILL MAKE THE HEATMAP

import geopandas
import shapely
from shapely.geometry import Point

restPandI.drop(['Unnamed: 0'],axis=1, inplace=True)

restPandI['geometry'] = restPandI[['Longitude', 'Latitude']].values.tolist()
# Change the coordinates to a geoPoint
restPandI['geometry'] = restPandI['geometry'].apply(Point)
restGEO = geopandas.GeoDataFrame(restPandI, geometry='geometry')
restGEO.crs= "+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs"
restGEO = restGEO.to_crs({'init':'epsg:3857'})
# REMOVE THE DAMN NULLS AND NANS

restGEO.dropna() 
#restGEO = restGEO[restGEO['SCORE'].notna()]
#del restGEO['VIOLATION DESCRIPTION']
restGEO.to_file('../shapefiles/restPandIGEO.shp', driver='Shapefile',encoding="utf-8")
restGEO.isnull().sum()
#restGEO.info
restGEO

/Users/RoscoeBColtrane/anaconda3/lib/python3.7/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


,Longitude,Latitude,Name,BORO,BUILDING,STREET,ZIPCODE,CUISINE DESCRIPTION,SCORE,VIOLATION DESCRIPTION,GRADE,geometry
0,-73.891889,40.824336,LITTLE CAESARS,Bronx,1054,SOUTHERN BOULEVARD,10459.0,Pizza,7.0,Non-food contact surface improperly constructe...,A,POINT (-8225607.424 4986465.763)
1,-74.032595,40.619722,NINO'S PIZZA,Brooklyn,9110,3 AVENUE,11209.0,Pizza,47.0,Evidence of mice or live mice present in facil...,NaN,POINT (-8241270.827 4956411.531)
2,-74.010325,40.648612,CAFE LORE,Brooklyn,4601,4 AVENUE,11220.0,Italian,34.0,"Toxic chemical improperly labeled, stored or u...",NaN,POINT (-8238791.639 4960649.403)
3,-73.947906,40.804435,RISTORANTE SETTEPANI,Manhattan,196,LENOX AVENUE,10026.0,Italian,11.0,Proper sanitization not provided for utensil w...,A,POINT (-8231843.211 4983538.524)
4,-73.959227,40.767632,BOTTEGA RESTAURANT,Manhattan,1331,2 AVENUE,10021.0,Italian,22.0,Non-food contact surface improperly constructe...,NaN,POINT (-8233103.503 4978127.683)
...,...,...,...,...,...,...,...,...,...,...,...,...
31166,-73.888801,40.657246,PAPA JOHN'S,Brooklyn,529,STANLEY AVE,11207.0,Pizza,NaN,Manufacture of frozen dessert not authorized o...,NaN,POINT (-8225263.756 4961916.229)
31167,-74.004713,40.741869,CAPPONE'S,Manhattan,75,9TH AVE,10011.0,Italian,19.0,Food not protected from potential source of co...,NaN,POINT (-8238166.968 4974341.640)
31168,-73.912157,40.613199,DAGAN PIZZA,Brooklyn,6187,STRICKLAND AVENUE,11234.0,Pizza,13.0,Facility not vermin proof. Harborage or condit...,A,POINT (-8227863.666 4955454.903)
31169,-73.878323,40.872737,DOMINO'S,Bronx,315,E 204TH ST,10467.0,Pizza,12.0,Facility not vermin proof. Harborage or condit...,A,POINT (-8224097.244 4993588.533)


In [17]:
subwayGEO = pd.read_csv("../data/DOITT_SUBWAY_STATION_01_13SEPT2010.csv", index_col=0)
print(subwayGEO.shape)
subwayGEO.to_csv('subwayGEO.csv')


(473, 5)


In [51]:
# BRING IN SUBWAYS FOR THE POINT OVERLAY

#subwayGEO.isnull().sum()
subwayGEO = pd.read_csv('subwayGEO.csv')
subwayGEO['coords']= subwayGEO['the_geom'].str.extract('.*\((.*)\).*')
# Create two lists for the loop results to be placed
lat = []
lon = []

# For each row in a varible,
for row in subwayGEO['coords']:
    # Try to,
    try:
        # Split the row by comma and append
        # everything before the comma to lon
        lon.append(row.split(' ')[0])
        # Split the row by comma and append
        # everything after the comma to at
        lat.append(row.split(' ')[1])
    # But if you get an error
    except:
        # append a missing value to lat
        lat.append(np.NaN)
        # append a missing value to lon
        lon.append(np.NaN)

# Create two new columns from lat and lon
subwayGEO['Latitude'] = lat
subwayGEO['Longitude'] = lon
subwayGEO

,URL,OBJECTID,NAME,the_geom,LINE,NOTES,coords,Latitude,Longitude
0,http://web.mta.info/nyct/service/,1,Astor Pl,POINT (-73.99106999861966 40.73005400028978),4-6-6 Express,"4 nights, 6-all times, 6 Express-weekdays AM s...",-73.99106999861966 40.73005400028978,40.73005400028978,-73.99106999861966
1,http://web.mta.info/nyct/service/,2,Canal St,POINT (-74.00019299927328 40.71880300107709),4-6-6 Express,"4 nights, 6-all times, 6 Express-weekdays AM s...",-74.00019299927328 40.71880300107709,40.71880300107709,-74.00019299927328
2,http://web.mta.info/nyct/service/,3,50th St,POINT (-73.98384899986625 40.76172799961419),1-2,"1-all times, 2-nights",-73.98384899986625 40.76172799961419,40.76172799961419,-73.98384899986625
3,http://web.mta.info/nyct/service/,4,Bergen St,POINT (-73.97499915116808 40.68086213682956),2-3-4,"4-nights, 3-all other times, 2-all times",-73.97499915116808 40.68086213682956,40.68086213682956,-73.97499915116808
4,http://web.mta.info/nyct/service/,5,Pennsylvania Ave,POINT (-73.89488591154061 40.66471445143568),3-4,"4-nights, 3-all other times",-73.89488591154061 40.66471445143568,40.66471445143568,-73.89488591154061
...,...,...,...,...,...,...,...,...,...
468,http://web.mta.info/nyct/service/,469,Coney Island - Stillwell Av,POINT (-73.9812359981396 40.57728100006751),D-F-N-Q,"D,F,N,Q-all times",-73.9812359981396 40.57728100006751,40.57728100006751,-73.9812359981396
469,http://web.mta.info/nyct/service/,470,34th St - Hudson Yards,POINT (-74.00219709442206 40.75544635961596),7-7 Express,"7-all times, 7 Express-rush hours AM westbound...",-74.00219709442206 40.75544635961596,40.75544635961596,-74.00219709442206
470,http://web.mta.info/nyct/service/,641,72nd St,POINT (-73.95836178682246 40.76880251014895),Q,Q-all times,-73.95836178682246 40.76880251014895,40.76880251014895,-73.95836178682246
471,http://web.mta.info/nyct/service/,642,86th St,POINT (-73.95177090964917 40.77786104333163),Q,Q-all times,-73.95177090964917 40.77786104333163,40.77786104333163,-73.95177090964917


In [58]:
#subwayGEO['geometry'] = restPandI[['Longitude', 'Latitude']].values.tolist()
# Change the coordinates to a geoPoint
subwayGEO['the_geom'] = restPandI['geometry'].apply(Point)
subwayGEO = gpd.GeoDataFrame(subwayGEO, geometry='the_geom')

from shapely import wkt
#subwayGEO['the_geom'] = subwayGEO['the_geom'].apply(wkt.loads)
subwayGEO = gpd.GeoDataFrame(subwayGEO, geometry='the_geom')

subwayGEO.crs= "+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs"
subwayGEO = subwayGEO.to_crs({'init':'epsg:3857'})
subwayGEO.crs

/Users/RoscoeBColtrane/anaconda3/lib/python3.7/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


{'init': 'epsg:3857'}

In [66]:
locations = subwayGEO[['Latitude', 'Longitude']]
locationlist = locations.values.tolist()

In [12]:

restPI_no_null = restPandI.dropna() 
restPI_no_null = restPI_no_null.dropna(subset = ['SCORE'],how='all') 
#del restPandI['VIOLATION DESCRIPTION']
#del restPandI['GRADE']
restPandI.to_csv('restPI_no_null.csv')
restPI_no_null.isnull().sum()
#restGEO.info

Longitude                0
Latitude                 0
Name                     0
BORO                     0
BUILDING                 0
STREET                   0
ZIPCODE                  0
CUISINE DESCRIPTION      0
SCORE                    0
VIOLATION DESCRIPTION    0
GRADE                    0
geometry                 0
dtype: int64

In [52]:
# Initialize a folium map and see what the default crs data is in case we need to convert the shapefile we loaded
distric_map = folium.Map(location=[40.7306,-73.9352], zoom_start=9, tiles='cartodbpositron' ) 
print('default map crs: ',distric_map.crs)


default map crs:  EPSG3857


In [59]:

folium.GeoJson(subwayGEO).add_to(distric_map)
distric_map

#Latitude	40.730610
#Longitude	-73.935242


/Users/RoscoeBColtrane/anaconda3/lib/python3.7/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


### THIS WAY LOOKS A BIT WEIRD BECAUSE THERE ARE SO MANY OVERLAID POINTS.

In [100]:
import pandas as pd 
import folium
from folium.plugins import HeatMap

for_map = restPI_no_null


hmap = folium.Map(location=[40.7306,-73.9352], zoom_start=11, tiles='cartodbpositron')

hm_wide = HeatMap( list(zip(for_map.Latitude.values, for_map.Longitude.values)),
                   min_opacity=0.2,
                   radius=17, blur=15, 
                   max_zoom=1, 
                 )

folium.GeoJson(subwayGEO).add_to(hmap)

#And then to save the generated leaflet map to file: 
hmap.add_child(hm_wide)


/Users/RoscoeBColtrane/anaconda3/lib/python3.7/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


In [64]:
hmap.save('../data/heatmapSubway.html')

In [96]:
### NOW WE WILL CLUSTER THE POINTS FOR A CLEANER LOOK.  ZOOM IN TO SEE MORE THEN CLICK A POINT OT GET THE NAME TO POP UP!

In [97]:
# CONTROL THE MAP

#from folium.element import IFrame
import folium.plugins
folium.plugins.MarkerCluster()
from folium import IFrame

text = 'A heatmap plot of all Pizza Joint locations with Subway Station call outs and popups with their name'
for_map = restPI_no_null


hmap = folium.Map(location=[40.7306,-73.9352], zoom_start=11, tiles='CartoDB dark_matter')

hm_wide = HeatMap( list(zip(for_map.Latitude.values, for_map.Longitude.values)),
                   min_opacity=0.2,
                   radius=17, blur=15, 
                   max_zoom=1, 
                 )

#folium.GeoJson(subwayGEO).add_to(hmap)

marker_cluster = folium.plugins.MarkerCluster().add_to(hmap)

for point in range(0, len(locationlist)):
    folium.Marker(locationlist[point], popup=subwayGEO['NAME'][point]).add_to(marker_cluster)

#And then to save the generated leaflet map to file: 
hmap.add_child(hm_wide)



In [84]:
hmap.save('../data/heatmap_Subway_Groups.html')

In [ ]:
# THIS CELL DOESNT RUN PROPERLY.  
# IT IS IHERE AS AN EXAMPLE FOR HOW TO DO THIS ANOTHER WAY WITH ONLY ONE LAYER OF DATA
import folium
import pandas as pd
 
NY_COORDINATES = (40.7306,-73.9352)
crimedata = subway
 
# for speed purposes
MAX_RECORDS = 1000
  
# create empty map zoomed in on San Francisco
map = folium.Map(location=NY_COORDINATES, zoom_start=11)
 
# add a marker for every record in the filtered data, use a clustered view
for each in subway[0:len(subway)].iterrows():
    map.simple_marker(
        location = [each[1]['Latitude'],each[1]['Longitude']], 
        clustered_marker = True)
  
display(map)